In [29]:
from utlities.cal_utils import Format
from utlities.cal_utils import Analyze

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker  # Import ticker module

import seaborn as sns
import math
import scipy
from scipy.signal import find_peaks
import numpy as np
import os

rep = "3"

path_data = f"./Data/rep{rep}_data.xlsx"
path_md = f"./Data/rep{rep}_md.xlsx"


folder = f"./picker_outputs_rep{rep}"

option_graphs_folder_for_11mm = f"rep{rep}_11mM_option_graphs"
complete_graphs = f"rep{rep}_11mM_complete__graphs"

form = Format(path = path_data)
analyze = Analyze()

sns.set_style("darkgrid")
sns.color_palette('colorblind')


[(0.00392156862745098, 0.45098039215686275, 0.6980392156862745),
 (0.8705882352941177, 0.5607843137254902, 0.0196078431372549),
 (0.00784313725490196, 0.6196078431372549, 0.45098039215686275),
 (0.8352941176470589, 0.3686274509803922, 0.0),
 (0.8, 0.47058823529411764, 0.7372549019607844),
 (0.792156862745098, 0.5686274509803921, 0.3803921568627451),
 (0.984313725490196, 0.6862745098039216, 0.8941176470588236),
 (0.5803921568627451, 0.5803921568627451, 0.5803921568627451),
 (0.9254901960784314, 0.8823529411764706, 0.2),
 (0.33725490196078434, 0.7058823529411765, 0.9137254901960784)]

In [30]:
baseline = 40

df = form.tidy_file(delta = False, average = True, baseline = baseline, errors= True)
df

Normalization issue for 2.5wt% RTG NoInh - Islet 4 Cell 1 because values in first 40 seconds are zero. 
                            Time frame would need to be expanded to 200 seconds. Values will be replaced with #DIV/0!
Normalization issue for 2.5wt% RTG NoInh - Islet 4 Cell 3 because values in first 40 seconds are zero. 
                            Time frame would need to be expanded to 140 seconds. Values will be replaced with #DIV/0!
Normalization issue for 2.5wt% RTG NoInh - Islet 4 Cell 5 because values in first 40 seconds are zero. 
                            Time frame would need to be expanded to 120 seconds. Values will be replaced with #DIV/0!
Normalization issue for 2.5wt% RTG NoInh - Islet 4 Cell 6 because values in first 40 seconds are zero. 
                            Time frame would need to be expanded to 110 seconds. Values will be replaced with #DIV/0!


,Time (sec),[hr]:[min]:[sec],A.U.,Normalized A.U.,Cell #,Treatment
0,1.295,00:00:01.295000,1.889,0.824171,Cell 1,NoRTG NoInh - Islet 1
1,2.59,00:00:02.590000,2.333,1.017888,Cell 1,NoRTG NoInh - Islet 1
2,3.885,00:00:04,2.222,0.969459,Cell 1,NoRTG NoInh - Islet 1
3,5.18,00:00:05.180000,2.222,0.969459,Cell 1,NoRTG NoInh - Islet 1
4,6.475,00:00:06.475000,2.333,1.017888,Cell 1,NoRTG NoInh - Islet 1
...,...,...,...,...,...,...
455420,2517.48,00:41:57.480000,3.3315,0.560198,AVG,10wt% RTG NoInh - Islet 2
455421,2518.775,00:41:58.775000,3.166625,0.532474,AVG,10wt% RTG NoInh - Islet 2
455422,2520.07,00:42:00.070000,3.26725,0.549395,AVG,10wt% RTG NoInh - Islet 2
455423,2521.365,00:42:01.365000,3.1995,0.538002,AVG,10wt% RTG NoInh - Islet 2


In [31]:
df.columns

Index(['Time (sec)', '[hr]:[min]:[sec]', 'A.U.', 'Normalized A.U.', 'Cell #',
       'Treatment'],
      dtype='object')

In [32]:
df_complete = df.copy()
df_partial = df_complete[df_complete["Normalized A.U."] != "DIV/0!"]

all_cells = df_complete["Treatment"] + df_complete["Cell #"]
valid_cells = df_partial["Treatment"] + df_partial["Cell #"]

len(all_cells.unique()) - len(valid_cells.unique())

4

In [33]:
def wide_format(tidy_df, info = "Normalized A.U."):
    
    dfs = [tidy_df[["Time (sec)","[hr]:[min]:[sec]"]].reset_index(drop=True)]
    print(tidy_df[["Time (sec)","[hr]:[min]:[sec]"]])
    cells = tidy_df["Cell #"].unique()
    
    for cell in cells:
    
        cell_df = tidy_df[tidy_df["Cell #"] == cell][info].copy().reset_index(drop=True)
        
        dfs.append(pd.DataFrame({cell: cell_df}))

    return pd.concat(dfs, axis=1)


In [34]:
filename = f'Rep{rep}_{baseline}sec_Normalization_Ca+_data.xlsx'
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    # Loop through the dictionary and write each DataFrame to a different sheet
    for sheet_name in df["Treatment"].unique():
        tidy_df = df[df["Treatment"] == sheet_name]
        wide_df = wide_format(tidy_df, info = "Normalized A.U.")
        wide_df.to_excel(writer, sheet_name=sheet_name)


      


      Time (sec) [hr]:[min]:[sec]
0          1.295  00:00:01.295000
1           2.59  00:00:02.590000
2          3.885         00:00:04
3           5.18  00:00:05.180000
4          6.475  00:00:06.475000
...          ...              ...
54965    3089.87  00:51:29.870000
54966   3091.165  00:51:31.165000
54967    3092.46  00:51:32.460000
54968   3093.755  00:51:33.755000
54969    3095.05  00:51:35.050000

[54970 rows x 2 columns]
      Time (sec) [hr]:[min]:[sec]
54970      1.295  00:00:01.295000
54971       2.59  00:00:02.590000
54972      3.885         00:00:04
54973       5.18  00:00:05.180000
54974      6.475  00:00:06.475000
...          ...              ...
93863   2793.315  00:46:33.315000
93864    2794.61  00:46:34.610000
93865   2795.905  00:46:35.905000
93866     2797.2  00:46:37.200000
93867   2798.495  00:46:38.495000

[38898 rows x 2 columns]
       Time (sec) [hr]:[min]:[sec]
93868       1.295  00:00:01.295000
93869        2.59  00:00:02.590000
93870       3.885         0

In [ ]:


import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill

# Open the Excel file with openpyxl to format cells containing "DIV/0!"
wb = openpyxl.load_workbook(filename)
red_fill = PatternFill(start_color="ff6767", end_color="ff6767", fill_type="solid")

for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
    for row in ws.iter_rows():
        for cell in row:
            if cell.value == "DIV/0!":
                cell.fill = red_fill

wb.save(filename)